# Interoperability between Xproj and 3rd-party Xarray geospatial extensions

3rd-party geospatial extensions may leverage `xproj` in different ways:

- simply consume the API exposed via the `.proj` Dataset / DataArray accessor
- register a custom Xarray Dataset / DataArray accessor that implements one or more CRS-specific callbacks (example below)

In [1]:
import pyproj
import xarray as xr
import xproj

xr.set_options(display_expand_indexes=True);

## Register an Xarray "geo" accessor

Here below is a basic example of a custom Xarray Dataset accessor that is also explictly registered as a "geo" accessor via the `xproj.register_geoaccessor` class decorator. Important note: the latter must be applied after (on top of) the Xarray register decorators.

Registering this "geo" accessor allows executing custom logic from within the accessor (via callback methods like `__proj_set_crs__`) when calling `xproj` API.

In [2]:
@xproj.register_geoaccessor
@xr.register_dataset_accessor("geo")
class GeoAccessor:

    def __init__(self, obj):
        self._obj = obj
        self._crs = None

    @property
    def crs(self):
        return self._crs

    def __proj_set_crs__(self, crs_coord_name, crs):
        self._crs = crs
        print(f"changed CRS of {crs_coord_name!r} in GeoAccessor!")

        return self._obj


Let's see if it works as expected.

In [3]:
# create an empty dataset, The `.geo.crs` property is uninitialized

ds = xr.Dataset()

ds.geo.crs is None

True

In [4]:
# initialize the CRS via `.proj.assign_crs()`

ds_wgs84 = ds.proj.assign_crs(spatial_ref=pyproj.CRS.from_user_input("epsg:4326"))

changed CRS of 'spatial_ref' in GeoAccessor!


In [5]:
ds_wgs84

<xarray.Dataset> Size: 8B
Dimensions:      ()
Coordinates:
  * spatial_ref  int64 8B 0
Data variables:
    *empty*
Indexes:
    spatial_ref  CRSIndex (crs=EPSG:4326)

In [6]:
# The CRS should have been initialized in the `.geo` accessor as well!

ds_wgs84.geo.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich